In [1]:
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

In [2]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt 


In [7]:
#display(data)
def filter_data(data):
    
    max_flux = -12
    min_flux = 26
    data = data[data['flux_aper']<max_flux]

    data = data[data['significance']>2]
    data = data.sample(frac=1)

    data_class = data[['class']]

    data_sig = data['significance']
    data_id = data['src_id']
    data_name = data['src_n']
    obs_info_params = [ 'livetime','significance','likelihood','pileup_flag','mstr_sat_src_flag','mstr_streak_src_flag'   ,'gti_obs' , 'flux_significance_b'  , 'flux_significance_m' , 'flux_significance_s' , 'flux_significance_h' , 'flux_significance_u'    ]
    data_val = data.drop(columns=obs_info_params)
    flux_params = ['photflux_aper' ,'photflux_aper_lolim' , 'photflux_aper_hilim' , 'flux_aper' , 'flux_aper_lolim' , 'flux_aper_hilim' ]
    flux_params_band = []
    filters = ['_b' ,'_h' ,'_m' ,'_u' ,'_s']
    for fp in flux_params:
        for f in filters:
            temp = fp+f
            flux_params_band.append(temp)

    data_val = data_val.drop(columns=['index' , 'class'])

    for d in data_val:
        
        
        max_val = np.amax(data_val[d])
        min_val = np.amin(data_val[d]) 
        data_val[d] = (data_val[d]-min_val)/(max_val-min_val)
        mean = np.mean(data_val[d])
        var = np.var(data_val[d])
#y = (x - min) / (max - min)
        #data_val[d] = (data_val[d]-mean)/np.sqrt(var)
        #data_val[d] = data_val[d] / np.amax(data_val[d])
        
        
    data_val = data_val.replace(np.nan , 0)
    #msno.matrix(data_val)
    #plt.show()
    return data , data_val , data_class , data_sig , data_id , data_name

In [8]:
def filter_data(data_sent):
    data = data_sent.copy()
    max_flux = -12
    min_flux = 26
    data = data[data['flux_aper']<max_flux]

    data = data[data['significance']>2]
    data_class = data[['class']]

    data_sig = data['significance']
    data_id = data['src_id']
    data_name = data['src_n']
    obs_info_params = [ 'livetime','likelihood','pileup_flag','mstr_sat_src_flag','mstr_streak_src_flag'   ,'gti_obs' , 'flux_significance_b'  , 'flux_significance_m' , 'flux_significance_s' , 'flux_significance_h' , 'flux_significance_u'    ]
    data_val = data.drop(columns=obs_info_params).reset_index(drop=True)
    return data_val


In [9]:

def extract_data(data_sent , impute_fn,reduce_fn):
    data = data_sent.copy()
    data = data.sample(frac=1)
    data = filter_data(data)
    #display(data)
    data_id = data[['class' ,'src_n' , 'src_id' ,'significance']]
    data_id = data_id.reset_index(drop=True)
    data_val = data.drop(['index' , 'class' ,'src_n' , 'src_id' ,'significance'] , axis=1)
    data_val = reduce_fn(data_val)
    data_val = impute_fn(data_val)
    data_val = data_val.reset_index(drop=True)
    data_reduced = pd.concat([data_id , data_val] , axis=1)
    return(data_reduced)

def no_impute(data_sent):
    return data_sent

def median_impute(data_send):
    data = data_send.copy()
    for d in data:
        data.fillna((data[d].median()), inplace=True)
    return data

def zero_impute(data_sent):
    data= data_sent.copy()
    data = data.replace(np.nan , 0)
    return data

def norm_data(data_sent):
    data = data_sent.copy()
    #data.replace()
    for d in data:
        max_val = np.amax(data[d])
        min_val =  np.amin(data[d])
        data[d] = (data[d]-min_val)/(max_val-min_val)
    return data


def do_nothing(data_sent):
    return data_sent

def std_data(data_sent):
    data = data_sent.copy()
    for d in data:
        mean =  np.mean(data[d])
        std = np.sqrt(np.var(data[d]))
        data[d] = (data[d]-mean)/std 
    return data


In [5]:

train_bh = pd.read_csv('processed_data/BH_.csv')
train_bh = train_bh.sample(frac=1)

train_ns = pd.read_csv('processed_data/NS_.csv')
train_ns = train_ns.sample(frac=1)

train = pd.concat([train_bh , train_ns ] , axis=0)
train =  train.sample(frac=1)


In [10]:
norm_data = extract_data(train , median_impute , norm_data).reset_index(drop=True)
norm_data.index.name = 'index'
display(norm_data.describe())
norm_data.to_csv('processed_data/train_norm_median_impute')

,significance,flux_aper_hilim_s,flux_aper_hilim_u,flux_aper_hilim_m,flux_aper_hilim_h,flux_aper_hilim_b,flux_aper_lolim_s,flux_aper_lolim_u,flux_aper_lolim_m,flux_aper_lolim_h,...,flux_brems,flux_brems_lolim,flux_brems_hilim,brems_kt,brems_kt_hilim,brems_kt_lolim,brems_nh,brems_nh_hilim,brems_nh_lolim,brems_stat
count,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,...,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000
mean,70.680500,0.432186,0.399933,0.436436,0.486806,0.502991,0.471289,0.399244,0.456698,0.482287,...,0.366822,0.365752,0.367300,0.354110,0.361636,0.349912,0.354869,0.355594,0.356215,0.355732
std,109.946267,0.193170,0.166272,0.173786,0.185332,0.185008,0.169059,0.085081,0.167799,0.173541,...,0.111591,0.111370,0.111445,0.128678,0.106576,0.122419,0.120236,0.121704,0.116295,0.108540
min,2.110000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11.120000,0.309677,0.297512,0.313406,0.358979,0.368118,0.391946,0.391946,0.391382,0.391946,...,0.391946,0.391946,0.391946,0.391946,0.391946,0.391946,0.391946,0.391946,0.391946,0.391946
50%,23.470000,0.391946,0.391946,0.402127,0.502370,0.504328,0.391946,0.391946,0.391946,0.437503,...,0.391946,0.391946,0.391946,0.391946,0.391946,0.391946,0.391946,0.391946,0.391946,0.391946
75%,49.260000,0.555333,0.531809,0.533815,0.615238,0.626640,0.549342,0.391946,0.552846,0.599765,...,0.391946,0.391946,0.391946,0.391946,0.391946,0.391946,0.391946,0.391946,0.391946,0.391946
max,376.060000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:

norm_data = extract_data(train , zero_impute , norm_data).reset_index(drop=True)
norm_data.index.name = 'index'
display(norm_data.describe())
norm_data.to_csv('processed_data/train_norm')

std_data = extract_data(train , zero_impute , std_data).reset_index(drop=True)
std_data.index.name = 'index'
display(std_data.describe())
std_data.to_csv('processed_data/train_std')

og_data = extract_data(train , zero_impute , do_nothing).reset_index(drop=True)
og_data.index.name = 'index'
display(og_data.describe())
og_data.to_csv('processed_data/og_norm')



TypeError: 'DataFrame' object is not callable